# **Atividade Computacional 02 - TEEC III**
## Aluno: André Paiva Conrado Rodrigues

### Importando bibliotecas

Nesta seção do código, são importadas bibliotecas de manipulação de dados importantes para a obtenção das amostras dos arquivos de áudio, bem como para a manipulação das mesmas.


In [ ]:
import numpy as np
import math
from scipy import signal, fft, ifft
from scipy.io import wavfile
from IPython.display import Audio
import matplotlib.pyplot as plt

In [ ]:
def separa_stereo(data_np_array):
    cont = 0
    tmpListDataL = []
    tmpListDataR = []
    while cont < data_np_array.shape[0]:
        tmpListDataL.append(data_np_array[cont][0])
        tmpListDataR.append(data_np_array[cont][1])
        cont = cont + 1
    dataL = np.array(tmpListDataL)
    dataR = np.array(tmpListDataR) 
    return dataL, dataR

### Traçando espectrogramas

No trecho de código abaixo, é definida a função `spectro`, que tem por função geral traçar o espectrograma referente a um determinado arquivo com uma determinada largura de janelamento, ambos parâmetros definidos pelo usuário.

Também é definida uma função auxiliar denominada `spectroPlot`, a qual é chamada na função `spectro`, que tem por função plotar o gráfico do espectrograma propriamente dito.

Na função `spectro`, é distinguido se o arquivo escolhido pelo usuário é mono ou stereo. Caso o arquivo seja stereo, os dois canais de áudio são separados e duas plotagens serão efetuadas, uma para cada canal. Caso o arquivo seja mono, é feita apenas uma plotagem.

In [ ]:
def spectroPlot(audioArray, rate, windowSize, vmin, vmax, title):
    f, t, Sxx = signal.spectrogram(audioArray, rate, nperseg=windowSize)
    plt.figure(figsize=(20,5))
    plt.pcolormesh(t, f, np.abs(Sxx), shading='flat', cmap='PRGn', vmin=vmin, vmax=vmax)
    plt.title(title)
    plt.ylabel('Frequência [Hz]')
    plt.xlabel('Tempo [s]')
    plt.colorbar()
    plt.show()
    
def spectro(file, windowSize):
    filename = file
    vmin = 0
    vmax = 1023
    samplerate, data = wavfile.read(filename)
    qtdSamples = data.shape[0]
    if(data.ndim == 2):
        dataL, dataR = separa_stereo(data)  
    if(data.ndim == 2):
        spectroPlot(dataL, samplerate, windowSize, vmin, vmax, 'Espectrograma do Canal L')
        spectroPlot(dataR, samplerate, windowSize, vmin, vmax, 'Espectrograma do Canal R')
    else:
        spectroPlot(data, samplerate, windowSize, vmin, vmax, 'Espectrograma do Sinal')
    

### Plotagem de espectrograma de arquivo mono

In [ ]:
file = 'guitar.wav'
spectro(file, 512)

### Plotagem de espectrogramas de arquivo stereo

In [ ]:
file = 'cover_altj_intro.wav'
spectro(file, 1024)

### Função de detecção de On-sets (ODF)

In [ ]:
def ODF_HFC_CALC(data_np_array, windowSize, overlap):
    sizerange = np.arange(1, windowSize+1)
    hfc_calc = []
    i = 0
    while i < data_np_array.shape[0]:
        datatmp = data_np_array[i : i + windowSize]
        ffttmp = fft.fft(datatmp, n=windowSize)
        ffttmp = abs(sizerange)*(abs(ffttmp)**2)
        hfc_calc.append(np.sum(abs(ffttmp[0:int(windowSize/2)]))*(2/windowSize))
        i = i + int(windowSize*(1-overlap))
    return hfc_calc

In [ ]:
def ODF_SD_CALC(data_np_array, windowSize, overlap):
    sizerange = np.arange(1, windowSize+1)
    sd_calc = []
    i = 0
    while i < data_np_array.shape[0]:
        indexOverlapStart = int(i + (windowSize*(1-overlap)))
        indexOverlapEnd = int(indexOverlapStart + windowSize)
        datatmp1 = data_np_array[i : i+windowSize]
        ffttmp1 = fft.fft(datatmp1, n=windowSize)
        datatmp2 = data_np_array[indexOverlapStart : indexOverlapEnd]
        ffttmp2 = fft.fft(datatmp2, n=windowSize)
        diftmp = abs(ffttmp1) - abs(ffttmp2)
        diftmp = ((diftmp + abs(diftmp))/2)**2
        sd_calc.append(np.sum(abs(diftmp[0:int(windowSize/2)])))
        i = i + int(windowSize*(1-overlap))
    return sd_calc

In [ ]:
def ODF_PLOT(x_axis, y_axis, tipo, canal="MONO"):
    plt.figure(figsize=(15,5))
    plt.margins(0, 0)
    graphcolor = 'g'
    title = "ODF - "
    if tipo == "HFC":
        title = title + "Método do conteúdo de alta frequência"
        plt.ylabel('Diferença energética')
    elif tipo == "SD":
        title = title + "Método da diferença espectral"
        plt.ylabel('Diferença espectral')
        graphcolor = 'y'
    if canal == "L":
        title = title + " - Canal Esquerdo"
    elif canal == "R":
        title = title + " - Canal Direito"    
    plt.xlabel('Tempo [s]')
    plt.title(title)
    plt.plot(y_axis, x_axis, graphcolor)
    plt.show()

In [ ]:
def ODF(filename, windowSize, overlap, tipo):
    samplerate, data = wavfile.read(filename)
    f = samplerate/2
    qtdSamples = data.shape[0]
    window = signal.windows.hann(windowSize)
    maxTimeAxis = qtdSamples/samplerate
    stepTimeAxis = (windowSize*(1-overlap))/samplerate
    timeAxis = np.arange(0.0, maxTimeAxis, stepTimeAxis)
    if(data.ndim == 2):
        dataL, dataR = separa_stereo(data)
        if tipo == 'HFC':
            hfcL = ODF_HFC_CALC(dataL, windowSize, overlap)
            ODF_PLOT(hfcL, timeAxis, 'HFC', 'L')
            hfcR = ODF_HFC_CALC(dataR, windowSize, overlap)
            ODF_PLOT(hfcR, timeAxis, 'HFC', 'R')
        elif tipo == 'SD':
            sdL = ODF_SD_CALC(dataL, windowSize, overlap)
            ODF_PLOT(sdL, timeAxis, 'SD', 'L')
            sdR = ODF_SD_CALC(dataR, windowSize, overlap)
            ODF_PLOT(sdR, timeAxis, 'SD', 'R')
    else:
        if tipo == 'HFC':
            hfc = ODF_HFC_CALC(data, windowSize, overlap)
            ODF_PLOT(hfc, timeAxis, 'HFC') 
        elif tipo == 'SD':
            sd = ODF_SD_CALC(data, windowSize, overlap)
            ODF_PLOT(sd, timeAxis, 'SD')  

In [ ]:
file = 'bat_jean_100bpm.wav'
ODF(file, 2048, 0.5, 'HFC')
ODF(file, 2048, 0.5, 'SD')

In [ ]:
file = 'GaitaTese.wav'
ODF(file, 4096, 0.5, 'HFC')
ODF(file, 4096, 0.5, 'SD')

In [ ]:
file = 'Only Sleeping100bpm(Violao).wav'
ODF(file, 16384, 0.5, 'HFC')
ODF(file, 16384, 0.5, 'SD')